# Configuration of Training Pipeline

In [12]:
from azureml.core import Workspace, Dataset, Environment, Experiment
from azureml.core.authentication import InteractiveLoginAuthentication
from azureml.core.compute import ComputeTarget
from azureml.pipeline.steps import PythonScriptStep
from azureml.core.runconfig import RunConfiguration
from azureml.pipeline.core import Pipeline

In [2]:

tenant_id = '178121b9-4c42-4e6f-8611-0350f0190053'
workspace_name = 'mlw-nairobitalk-a001'
subscription_id = '23529470-ba17-4d8a-9f0c-064e63a49c33'
resource_group = 'rg-nairobitalk-a001'
environment_name = 'env-pytorch'
training_compute_name = 'gpu-cluster'
dataset_name = 'LegoSimpsonsDS'

## Connect to workspace

In [3]:
# Setup Workspace
# Connect to workspace
credentials = InteractiveLoginAuthentication(tenant_id = tenant_id)
ws = Workspace(subscription_id=subscription_id,
                resource_group=resource_group,
                workspace_name=workspace_name,
                auth = credentials)

## Get Training Cluster & DataSet

In [4]:
training_cluster = ComputeTarget(workspace=ws, name=training_compute_name)
dataset = Dataset.get_by_name(workspace=ws, name=dataset_name)

## Configure Training Environment

In [14]:
pipeline_run_config = RunConfiguration()
pipeline_run_config.target = training_cluster
pipeline_run_config.environment = Environment.get(workspace=ws, name='AzureML-pytorch-1.7-ubuntu18.04-py37-cuda11-gpu', version=3)

## Configure Training Steps

In [19]:
train_step = PythonScriptStep(
    name = "Train Model",
    source_directory = 'train',
    script_name = 'train.py',
    runconfig = pipeline_run_config,
    arguments=['--num-epochs', 8, '--model-name', 'SimpsonsClassification', '--data-path', dataset.as_mount()]
)
pipeline_steps = [train_step]
pipeline = Pipeline(workspace=ws, steps=pipeline_steps)

## Configure and execute experiment

In [20]:
experiment = Experiment(workspace=ws, name='lego-simpsons-training')
pipeline_run = experiment.submit(pipeline, regenerate_outputs=True)
pipeline_run.wait_for_completion(show_output=True)

Created step Train Model [d80f292d][23a3b5ac-5357-4815-815f-d8628710a129], (This step will run and generate new outputs)
Submitted PipelineRun 2e55cfde-9ae1-4800-8426-6ac6c51a978e
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/2e55cfde-9ae1-4800-8426-6ac6c51a978e?wsid=/subscriptions/23529470-ba17-4d8a-9f0c-064e63a49c33/resourcegroups/rg-nairobitalk-a001/workspaces/mlw-nairobitalk-a001&tid=178121b9-4c42-4e6f-8611-0350f0190053
PipelineRunId: 2e55cfde-9ae1-4800-8426-6ac6c51a978e
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/2e55cfde-9ae1-4800-8426-6ac6c51a978e?wsid=/subscriptions/23529470-ba17-4d8a-9f0c-064e63a49c33/resourcegroups/rg-nairobitalk-a001/workspaces/mlw-nairobitalk-a001&tid=178121b9-4c42-4e6f-8611-0350f0190053
PipelineRun Status: Running


StepRunId: f7c66964-7cea-4472-8c47-1b5bdf71de02
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/f7c66964-7cea-4472-8c47-1b5bdf71de02?wsid=/subscriptions/23529470-ba17-4d8a-9f0c-064e63a49c

'Finished'